In [ ]:
#@title # 🛠️ Panel de Configuración de ComfyUI
#@markdown ---
#@markdown ### 📁 Gestión de Almacenamiento (Google Drive)
#@markdown Elige cómo quieres manejar tus archivos. Montar Drive permite que tus modelos y resultados no se borren al cerrar Colab.
MODO_DRIVE = "SOLO_LOCAL" #@param ["MONTAR", "DESMONTAR", "SOLO_LOCAL"]
USAR_DRIVE_PARA_COMFY = True #@param {type:"boolean"}
#@markdown > **Nota:** Si activas la casilla de arriba, ComfyUI se instalará en `/content/drive/MyDrive/AI/ComfyUI`. Esto ahorra tiempo en futuras sesiones ya que no tendrás que re-descargar todo.
RUTA_PERSONALIZADA_DRIVE = "AI/ComfyUI" #@param {type:"string"}

#@markdown ---
#@markdown ### 🔄 Actualizaciones y Mantenimiento
ACTUALIZAR_COMFY = True #@param {type:"boolean"}
#@markdown > **Actualizar:** Descarga la versión más reciente de ComfyUI desde GitHub automáticamente.

#@markdown ---
#@markdown ### 🧠 Instalación de Modelos e Inteligencia Artificial
#@markdown Selecciona qué modelos quieres pre-instalar para empezar a trabajar de inmediato.
INSTALAR_QWEN_VL = False #@param {type:"boolean"}
#@markdown > **Edición de imagenes Qwen VL/2509:** Instala los modelos necesarios para usar estos Workflows.
#@markdown >
#@markdown > Un modelo multimodal avanzado capaz de "entender" imágenes y editarlas siguiendo instrucciones complejas. Requiere nodos específicos que se instalarán automáticamente.
#@markdown >
#@markdown > <font color='red'>Nota: Este modelo es muy pesado por lo que las maquinas free de Google Colab no suelen aguantarlo. (Se detiene por falta de RAM).</font>

INSTALAR_SD15_BASE = False #@param {type:"boolean"}
#@markdown > **Stable Diffusion 1.5:** El estándar de oro para generación de imágenes. Incluye el modelo base y un VAE para mejorar colores y rostros.

INSTALAR_PROTOVISION_XL = False #@param {type:"boolean"}
INSTALAR_JUGGERNAUT_XL_HYPER = False #@param {type:"boolean"}
INSTALAR_JUGGERNAUT_X = False #@param {type:"boolean"}
INSTALAR_JUGGERNAUT_V8 = False #@param {type:"boolean"}

#@markdown ---
#@markdown ### 📦 Instalar Modelos Personalizados
#@markdown Ingresa la lista de modelos en formato JSON.
#@markdown > Puedes generar la cadena JSON con el formato correcto a través del siguiente proyecto de Google Apps Script:
#@markdown > ### [👉 Haz clic AQUÍ para abrir la herramienta](https://script.google.com/macros/s/AKfycbzmfmxfFRKvlnwlDuHNwOhWqEzYd90f1YKaQxSuFQ3o8HyQW_oHZZgPo9j4vUQlJ7ZI/exec)

MODELOS_PERSONALIZADOS = "[]" #@param {type:"string"}

#@markdown ---
#@markdown ### 🚀 Configuración de Ejecución

USAR_LOWVRAM = False #@param {type:"boolean"}
#@markdown > **📉 Low VRAM:** Actívalo si Colab se cierra inesperadamente o te quedas sin memoria. Ayuda a gestionar mejor los recursos.

USAR_CPU_ONLY = False #@param {type:"boolean"}
#@markdown > **🐌 Solo CPU:** Actívalo si no tienes acceso a una GPU. La generación será extremadamente lenta.

TIPO_TUNEL = "LOCALTUNNEL" #@param ["CLOUDFLARE", "LOCALTUNNEL"]
#@markdown > **🌐 Tipo de Túnel:** Selecciona el método de conexión para acceder a la interfaz web.

ARGUMENTOS_ADICIONALES = "" #@param {type:"string"}
#@markdown > **⌨️ Argumentos Adicionales:** Inserta comandos extra para modificar el comportamiento del programa.

from google.colab import drive
from pathlib import Path
import os
import torch

if not torch.cuda.is_available():
    if not USAR_CPU_ONLY:
        print("\n" + "!"*60)
        print("❌ ERROR CRÍTICO: NO SE DETECTÓ GPU")
        print("------------------------------------------------------------")
        print("ComfyUI requiere una GPU para funcionar. Actualmente estás usando CPU.")
        print("👉 Ve a: Entorno de ejecución > Cambiar tipo de entorno de ejecución")
        print("👉 Selecciona 'T4 GPU' en Acelerador de hardware.")
        print("!"*60 + "\n")
        raise RuntimeError("Debes seleccionar un entorno con GPU (T4) para continuar.")
    else:
        print("\n⚠️ EJECUTANDO EN MODO CPU: La generación de imágenes será muy lenta.")

if MODO_DRIVE == "MONTAR":
  drive.mount('/content/drive', force_remount=True)
elif MODO_DRIVE == "DESMONTAR":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

WORKSPACE = '/content/ComfyUI'
if MODO_DRIVE == "MONTAR" and USAR_DRIVE_PARA_COMFY:
    WORKSPACE = "/content/drive/MyDrive/" + RUTA_PERSONALIZADA_DRIVE

if not os.path.exists(WORKSPACE):
    print("-= Initial setup ComfyUI =-")
    !git clone https://github.com/comfyanonymous/ComfyUI {WORKSPACE}
else:
    if ACTUALIZAR_COMFY:
        print("-= Updating ComfyUI =-")
        %cd {WORKSPACE}
        !git pull

%cd {WORKSPACE}

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {WORKSPACE}



%cd {WORKSPACE}
# Install Aria2
!apt-get -y install -qq aria2


from google.colab import userdata

CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
print("Loaded API key:", "✅" if CIVITAI_API_TOKEN else "❌ Not found")

# Install custom nodes

def install_custom_node(url):
  %cd {WORKSPACE}/custom_nodes
  !git clone {url}

def downloadModel(url, filename = None):
  if 'huggingface.co' in url:
    if filename is None:
      filename = url.split('/')[-1]
      filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    if filename:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN} -o {filename}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN}


# install custom nodes
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager.git')
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Pack')


# download models
%cd ./models/checkpoints

# CyberRealistic Pony
downloadModel('https://civitai.com/api/download/models/2071650')

# Protovision XL
if INSTALAR_PROTOVISION_XL:
    downloadModel('https://civitai.com/api/download/models/201514')
# Juggernaut XL Hyper
if INSTALAR_JUGGERNAUT_XL_HYPER:
    downloadModel('https://civitai.com/api/download/models/471120')
# Juppernaut X
if INSTALAR_JUGGERNAUT_X:
    downloadModel('https://civitai.com/api/download/models/456194')
# Juggernaut v8
if INSTALAR_JUGGERNAUT_V8:
    downloadModel('https://civitai.com/api/download/models/288982')

%cd {WORKSPACE}



if INSTALAR_QWEN_VL:
    # 1. Configuración de carpetas y descargas de Qwen
    # ----------------------------------------------------------------

    # VAE
    %cd {WORKSPACE}/models/vae
    downloadModel('https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/vae/qwen_image_vae.safetensors')

    # DIFFUSION MODELS (Unet)
    # Nota: Si la carpeta no existe, la creamos
    !mkdir -p {WORKSPACE}/models/diffusion_models
    %cd {WORKSPACE}/models/diffusion_models
    downloadModel('https://huggingface.co/Comfy-Org/Qwen-Image-Edit_ComfyUI/resolve/main/split_files/diffusion_models/qwen_image_edit_fp8_e4m3fn.safetensors')

    # TEXT ENCODERS
    !mkdir -p {WORKSPACE}/models/text_encoders
    %cd {WORKSPACE}/models/text_encoders
    downloadModel('https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI/resolve/main/split_files/text_encoders/qwen_2.5_vl_7b_fp8_scaled.safetensors')

    # LORAS
    %cd {WORKSPACE}/models/loras
    downloadModel('https://huggingface.co/lightx2v/Qwen-Image-Lightning/resolve/main/Qwen-Image-Edit-Lightning-4steps-V1.0-bf16.safetensors')

    # 2. Instalación del Nodo Personalizado (Obligatorio para que funcione el workflow)
    # ----------------------------------------------------------------
    %cd {WORKSPACE}/custom_nodes
    # Clonamos el repositorio oficial que maneja estos modelos de Qwen
    !git clone https://github.com/Comfy-Org/ComfyUI-Qwen-VL-API.git 2>/dev/null || echo "🟢 Edición de imagen de Qwen/2095 - Instalacion Finalizada"

    # Volver al inicio
    %cd {WORKSPACE}

if INSTALAR_SD15_BASE:
    # --- Descarga de Modelos SD 1.5 (Optimizados) ---

    # SD1.5 Pruned (Versión Safetensors - Más rápida y segura)
    %cd {WORKSPACE}/models/checkpoints
    downloadModel('https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors')

    # AbyssOrangeMix2 (Anime Style)
    downloadModel('https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors')

    # VAE Standard (Mejora el color y detalles en SD 1.5)
    %cd {WORKSPACE}/models/vae
    downloadModel('https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors')

    %cd {WORKSPACE}

if MODELOS_PERSONALIZADOS:
    import json
    print("⬇️ Descargando modelos personalizados...")
    try:
        custom_models = json.loads(MODELOS_PERSONALIZADOS)
        if isinstance(custom_models, list):
            for model in custom_models:
                path_str = model.get("ruta", "").strip()
                url = model.get("link", "").strip()

                if path_str and url:
                    if "/" in path_str:
                        folder_part, filename_part = path_str.rsplit("/", 1)
                        folder_part = folder_part.strip()
                        filename_part = filename_part.strip()

                        target_path = f"{WORKSPACE}/models/{folder_part}"
                        !mkdir -p {target_path}
                        %cd {target_path}
                        downloadModel(url, filename_part)
                    else:
                        print(f"⚠️ Formato incorrecto en ruta: {path_str}")
    except json.JSONDecodeError:
        print("⚠️ Error: MODELOS_PERSONALIZADOS no es un JSON válido.")

    %cd {WORKSPACE}


!ls -al ./models/checkpoints/


if TIPO_TUNEL == "CLOUDFLARE":
    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb
elif TIPO_TUNEL == "LOCALTUNNEL":
    !npm install -g localtunnel



# --- CREACIÓN DEL SCRIPT LANZADOR PERSISTENTE ---
%cd {WORKSPACE}

comfy_args = ["python", "main.py", "--dont-print-server"]
if USAR_CPU_ONLY:
    comfy_args.append("--cpu")
if USAR_LOWVRAM:
    comfy_args.append("--lowvram")
if ARGUMENTOS_ADICIONALES:
    comfy_args.extend(ARGUMENTOS_ADICIONALES.split())

launcher_script = f"""
import subprocess
import threading
import time
import socket
import os
import urllib.request

def launch_tunnel(port):
    print("\\n[Túnel] Esperando a que ComfyUI inicie...")
    while True:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        if sock.connect_ex(('127.0.0.1', port)) == 0:
            sock.close()
            break
        sock.close()
        time.sleep(1)

    tunnel_type = "{TIPO_TUNEL}"

    if tunnel_type == "CLOUDFLARE":
        proc = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{{}}".format(port)],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        for line in proc.stderr:
            if "trycloudflare.com " in line:
                url = line.split("http")[1].strip()
                print("\\n" + "="*50)
                print(f"🔗 URL DE ACCESO: http{{url}}")
                print("="*50 + "\\n")

    elif tunnel_type == "LOCALTUNNEL":
        try:
            ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\\n")
            print("\\n" + "="*50)
            print(f"🔐 PASSWORD/IP LOCALTUNNEL: {{ip}}")
            print("="*50 + "\\n")
        except:
            pass

        proc = subprocess.Popen(["lt", "--port", "{{}}".format(port)],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        for line in proc.stdout:
            print(line, end='')
            if "your url is" in line:
                url = line.split("is:")[1].strip()
                print("\\n" + "="*50)
                print(f"🔗 URL DE ACCESO: {{url}}")
                print("="*50 + "\\n")

if __name__ == "__main__":
    threading.Thread(target=launch_tunnel, args=(8188,), daemon=True).start()
    subprocess.run({comfy_args})
"""

with open("colab_launcher.py", "w") as f:
    f.write(launcher_script)

print("\n🚀 Iniciando ComfyUI...")
# Ejecutar en segundo plano con nohup para liberar la celda
!nohup python -u colab_launcher.py > comfy.log 2>&1 &

print("⏳ Esperando URL de Cloudflare (esto puede tardar unos segundos)...")
import time
found = False
file_pos = 0
while not found:
    if os.path.exists("comfy.log"):
        with open("comfy.log", "r") as f:
            f.seek(file_pos)
            chunk = f.read()
            if chunk:
                print(chunk, end='', flush=True)
                file_pos = f.tell()
                if "trycloudflare.com" in chunk or "your url is" in chunk:
                    for line in chunk.splitlines():
                        if "trycloudflare.com" in line:
                            url = line.split("http")[1].strip()
                            print(f"\n✅ URL DE ACCESO: http{url}")
                            print("⚠️ La celda se cerrará, pero ComfyUI sigue funcionando en segundo plano.")
                            found = True
                            break
                        elif "your url is" in line:
                            url = line.split("is:")[1].strip()
                            print(f"\n✅ URL DE ACCESO: {url}")
                            print("⚠️ La celda se cerrará, pero ComfyUI sigue funcionando en segundo plano.")
                            found = True
                            break
    time.sleep(2)

In [ ]:
#@title # 📦 Instalar Modelos Personalizados Post-ejecución
#@markdown ---
#@markdown ## Si necesitas instalar modelos despues de la ejecucion princpal lo puedes hacer desde aca:
#@markdown Ingresa la lista de modelos en formato JSON en le recuadro de abajo, despues de colocar linea JSON ejecuta la celda.
#@markdown > Puedes generar la cadena JSON con el formato correcto a través del siguiente proyecto de Google Apps Script:
#@markdown > ### [👉 Haz clic AQUÍ para abrir la herramienta](https://script.google.com/macros/s/AKfycbzmfmxfFRKvlnwlDuHNwOhWqEzYd90f1YKaQxSuFQ3o8HyQW_oHZZgPo9j4vUQlJ7ZI/exec)

MODELOS_PERSONALIZADOS = "[{\"id\":1,\"ruta\":\"vae/ae.safetensors\",\"link\":\"https://huggingface.co/Comfy-Org/Lumina_Image_2.0_Repackaged/resolve/main/split_files/vae/ae.safetensors\",\"workflow\":\"image_omnigen2_image_edit\"},{\"id\":2,\"ruta\":\"text_encoders/qwen_2.5_vl_fp16.safetensors\",\"link\":\"https://huggingface.co/Comfy-Org/Omnigen2_ComfyUI_repackaged/resolve/main/split_files/text_encoders/qwen_2.5_vl_fp16.safetensors\",\"workflow\":\"image_omnigen2_image_edit\"},{\"id\":3,\"ruta\":\"diffusion_models/omnigen2_fp16.safetensors\",\"link\":\"https://huggingface.co/Comfy-Org/Omnigen2_ComfyUI_repackaged/resolve/main/split_files/diffusion_models/omnigen2_fp16.safetensors\",\"workflow\":\"image_omnigen2_image_edit\"}]" #@param {type:"string"}
#@markdown ---
#@markdown <font color='#888'>Nota: Asegúrate de haber ejecutado primero la celda principal para definir la ruta de `WORKSPACE`.</font>

import json
import os
from google.colab import userdata

def descargar_modelos_custom():
    if not MODELOS_PERSONALIZADOS or MODELOS_PERSONALIZADOS == "[]":
        print("ℹ️ No se ingresaron modelos para descargar.")
        return

    # Intentar cargar el token de Civitai si existe
    try:
        CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
    except:
        CIVITAI_API_TOKEN = ""
        print("⚠️ Civitai Token no configurado en 'Secrets'.")

    def download_logic(url, filename=None):
        auth_token = f"?token={CIVITAI_API_TOKEN}" if CIVITAI_API_TOKEN and "civitai.com" in url else ""
        output_cmd = f"-o {filename}" if filename else ""

        # Usamos aria2 para descargas rápidas y resilientes
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}{auth_token}" {output_cmd}

    print("⬇️ Iniciando descarga de modelos personalizados...")

    try:
        custom_models = json.loads(MODELOS_PERSONALIZADOS)
        if isinstance(custom_models, list):
            for model in custom_models:
                path_str = model.get("ruta", "").strip()
                url = model.get("link", "").strip()

                if path_str and url:
                    # Separar carpeta y nombre de archivo
                    if "/" in path_str:
                        folder_part, filename_part = path_str.rsplit("/", 1)
                    else:
                        folder_part = "checkpoints" # Carpeta por defecto
                        filename_part = path_str

                    # Construir ruta completa
                    target_path = os.path.join(WORKSPACE, "models", folder_part)
                    os.makedirs(target_path, exist_ok=True)

                    %cd {target_path}
                    print(f"📥 Descargando: {filename_part} en models/{folder_part}")
                    download_logic(url, filename_part)
                else:
                    print(f"⚠️ Saltando entrada inválida: {model}")

        print("\n✅ ¡Proceso de descarga finalizado!")
        %cd {WORKSPACE}

    except json.JSONDecodeError:
        print("❌ Error: El contenido de MODELOS_PERSONALIZADOS no es un JSON válido.")
    except NameError:
        print("❌ Error: La variable 'WORKSPACE' no está definida. Ejecuta primero la celda de instalación.")

descargar_modelos_custom()

In [ ]:
#@title 🖥️ Terminal Web Interactiva (ttyd)
#@markdown Esta celda abrirá una terminal en una pestaña nueva del navegador.

import os
import subprocess
import time
import urllib.request

def setup_terminal():
    print("📦 Instalando dependencias (ttyd y localtunnel)...")
    # Instalar ttyd
    !apt-get update -qq && apt-get install -y -qq ttyd > /dev/null
    # Instalar localtunnel si no está
    if not os.path.exists("/usr/local/bin/lt"):
        !npm install -g localtunnel > /dev/null

    # Puerto para la terminal
    PORT = 7681

    # 1. Obtener la IP pública (necesaria para la pantalla de seguridad de LocalTunnel)
    try:
        ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
    except:
        ip = "No se pudo obtener la IP"

    # 2. Iniciar ttyd en segundo plano
    # Usamos 'bash' para que sea una terminal completa
    subprocess.Popen(["ttyd", "-p", str(PORT), "bash"],
                     stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    print("⏳ Iniciando túnel...")
    time.sleep(2)

    # 3. Iniciar LocalTunnel
    print("\n" + "="*60)
    print(f"🔑 PASSWORD/IP DE ACCESO: {ip}")
    print("="*60)
    print("👉 Instrucciones:")
    print("1. Haz clic en el enlace 'url is:' que aparecerá abajo.")
    print(f"2. Pega la IP ({ip}) en la caja de texto que dice 'Endpoint IP'.")
    print("3. ¡Listo! Ya tienes control total de la máquina.")
    print("="*60 + "\n")

    # Ejecutar localtunnel
    !lt --port {PORT}

setup_terminal()